# **Data Collection Notebook**

## Objectives

- Fetch data from Kaggle and save as raw data.
- Inspect the data and save under outputs/datasets/collection

## Inputs

* Kaggle JSON file - authentication token 

## Outputs

* The Kaggle files were unzipped to:
* inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/house_prices_records.csv
* inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/inherited_houses.csv

 


---
## Change Directory

### 1. We need to change the working directory from its current folder to its parent folder
We access the current directory with os.getcwd()


In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/PP5-iowa-housing-market/jupyter_notebooks'

### 2. We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


### 3. Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspaces/PP5-iowa-housing-market'

---

## Import Packages

In [4]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from pandas_profiling import ProfileReport

## Load the House Price Records

In [5]:
df = pd.read_csv(f"outputs/datasets/collection/house_price_records.csv")
print(df.shape)
df.head()

(1460, 24)


,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,GarageYrBlt,GrLivArea,KitchenQual,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854.0,3.0,No,706,GLQ,150,0.0,548,RFn,2003.0,1710,Gd,8450,65.0,196.0,61,5,7,856,0.0,2003,2003,208500
1,1262,0.0,3.0,Gd,978,ALQ,284,NaN,460,RFn,1976.0,1262,TA,9600,80.0,0.0,0,8,6,1262,NaN,1976,1976,181500
2,920,866.0,3.0,Mn,486,GLQ,434,0.0,608,RFn,2001.0,1786,Gd,11250,68.0,162.0,42,5,7,920,NaN,2001,2002,223500
3,961,NaN,NaN,No,216,ALQ,540,NaN,642,Unf,1998.0,1717,Gd,9550,60.0,0.0,35,5,7,756,NaN,1915,1970,140000
4,1145,NaN,4.0,Av,655,GLQ,490,0.0,836,RFn,2000.0,2198,Gd,14260,84.0,350.0,84,5,8,1145,NaN,2000,2000,250000


---

## Data Exploration

Explore the dataset, by checking variable types and distribution, missing levels and what value these variables may add in the context of the first business requirement.

In [ ]:
pandas_report = ProfileReport(df=df, minimal=True)
pandas_report.to_notebook_iframe()

--- 

## Correlation Study

Assess correlation levels across numerical variables using spearman and pearson methods.

* We will exclude the first item returned as this will be the correlation between SalePrice and SalePrice
* We will only fetch the 10 most relevant correlations

### 1. Using the 'pearson' method to measure the linear relationship between two features

In [ ]:
corr_pearson = df.corr(method='pearson')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(10)
corr_pearson

### 2. Using the 'spearman' method to measure the linear relationship between two features

In [ ]:
corr_spearman = df.corr(method='spearman')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(10)
corr_spearman

### Notes

For both Spearman and Pearson methods, we found valuable correlation data between sales price and at least 5 variables.

We take the top 5 variables returned for each method, transform them into a list and concatenate both lists, in order to have a list with the most crrelated values.

In [ ]:
top_n = 5
set(corr_pearson[:top_n].index.to_list() + corr_spearman[:top_n].index.to_list())


The result is 6 variables that correlate to Sale Price, These variables will be tested on strength to predicting Sale Price

In [ ]:
corr_var_list = list(set(corr_pearson[:top_n].index.to_list() + corr_spearman[:top_n].index.to_list()))
corr_var_list

---

## EDA on the Correlated Variable List

Filter the house price dataset on only the correlated variable list and include the sale price

In [ ]:
df_eda = df.filter(corr_var_list + ['SalePrice'])
print(df_eda.shape)
df_eda.head(5)

## Variables Distribution by SalePrice

We plot the distribution (numerical and categorical) coloured by SalePrice

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

%matplotlib inline

def plot_numerical(df, col, target_var):
  plt.figure(figsize=(15, 8))
  sns.regplot(data=df, x=col, y=target_var)  
  plt.title(f"{col}", fontsize=20)
  plt.show()


target_var = 'SalePrice'
for col in corr_var_list:
  plot_numerical(df_eda, col, target_var)
  print("\n\n")

---
## Conclusions and Next Steps

The correlations and plots interpretation converge.

* The following are the variables isolated in the correlation study:
    * 1stFlrSF: First Floor square feet
    * GarageArea: Size of garage in square feet
    * GrLivArea: Above grade (ground) living area square feet
    * OverallQual: Rates the overall material and finish of the house
    * YearBuilt: Original construction date
    * TotalBsmtSF: Total square feet of basement area

The correlation analysis shows that the sizes of the ground floor living area, Basement area and the garage area, play a key role in determining house price. In addition, the year the house was built and the overall quality of materials used and the finishes in the house also play a significant role in determining house price.

The plots show that the variables, isolated in the correlation study, do indeed have a strong correlation and hence possibly strong predictive power for Sale Price


The next step is **Data Cleaning**

## Push files to Repo